In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install python-docx
!nvidia-smi
from tensorflow.keras.preprocessing.text import Tokenizer
import docx
import re
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import random
import numpy as np
import tensorflow as tf
import os

Wed May 19 11:53:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
class data_processing:
  
  def __init__(self,max_vocab , sequence_length):
    self.sequence_length = sequence_length
    self.max_vocab = max_vocab
    self.tokenizer = Tokenizer(num_words = self.max_vocab)
    self.text = self.getText("/content/drive/My Drive/sitharasongs.docx")

    self.lower_data = self.text.lower()           
    self.split_data = self.lower_data.splitlines()      
    self.final = ''     

    for line in self.split_data:
        line = self.clean_text(line)
        self.final += '\n' + line

    self.final_data = self.final.split('\n')       
    self.tokenizer.fit_on_texts(self.final_data)
    self.word2idx = self.tokenizer.word_index
    self.idx2word = dict(map(reversed, self.word2idx.items()))
    self.words = self.word2idx.keys()
    self.words_indexes = [self.word2idx[w] for w in self.words]
    self.len_word = len(self.words_indexes)

  def data_xy(self,index):    
      x = self.words_indexes[index : index + self.sequence_length]
      y = self.words_indexes[index + 1 : index + self.sequence_length+1]
      return {"X":x , "Y":y}
   
  def getText(self,filename):
      doc = docx.Document(filename)
      fullText = []
      for para in doc.paragraphs:
          fullText.append(para.text)
      return '\n'.join(fullText)

  def clean_text(self,text):
      text = re.sub(r',', '', text)
      text = re.sub(r'\'', '',  text)
      text = re.sub(r'\"', '', text)
      text = re.sub(r'\(', '', text)
      text = re.sub(r'\)', '', text)
      text = re.sub(r'\n', '', text)
      text = re.sub(r'“', '', text)
      text = re.sub(r'”', '', text)
      text = re.sub(r'’', '', text)
      text = re.sub(r'\.', '', text)
      text = re.sub(r';', '', text)
      text = re.sub(r':', '', text)
      text = re.sub(r'\-', '', text)
      return text

  def data_generator(self,batch_size = 10):
    while True:
      x_batch = np.empty((batch_size, self.sequence_length), dtype='int64') 
      y_batch = np.empty((batch_size, self.sequence_length), dtype='int64') 
      randomlist = random.sample(range(self.len_word-self.sequence_length), batch_size)
      for i in randomlist:
        data = self.data_xy(i)
        x_batch[:] = data["X"]
        y_batch[:] = data["Y"]
      yield (x_batch , y_batch)

In [4]:
class MyModel(tf.keras.Model):
  
  def __init__(self, vocab_size, embedding_dim):
    super().__init__(self)
    self.Embedding = Embedding(vocab_size, embedding_dim)
    self.Dropout = Dropout(0.2)
    self.Bidirectional = Bidirectional(layer=LSTM(340, return_sequences=True))
    self.LSTM = LSTM(100 , return_sequences=True)
    self.Dense_1 = Dense(1024, activation='relu')
    self.Dense_2 = Dense(vocab_size, activation='softmax')

  def call(self, inputs, return_state=False):
    x = inputs
    x = self.Embedding(x) 
    x = self.Bidirectional(x)
    x = self.Dropout(x)
    x = self.LSTM(x)
    x = self.Dense_1(x)
    x = self.Dropout(x)
    x = self.Dense_2(x)
    return x

In [21]:
class prediction:
  def __init__(self, model,tokenizer, idx2words ,temperature=1.0):
    self.temperature = temperature
    self.model = model
    self.tokenizer = tokenizer
    self.idx2word  = idx2word

  def pred_next_word(self, inputs):

    input_ids = self.tokenizer.word_index[inputs]
    input_ids = np.array([[input_ids]], dtype=np.float32)
    predicted_logits = self.model(inputs=input_ids)
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    predicted_chars = self.idx2word[int(np.squeeze(np.squeeze(predicted_ids)))]
    
    return predicted_chars

In [16]:
def train_model(max_vocab , sequence_length):

  df = data_processing(max_vocab=2101 , sequence_length = 20)
  sequence_length = 20
  EPOCHS = 5
  vocab_size = df.len_word
  model = MyModel(vocab_size,embedding_dim=256)
  loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

  checkpoint_dir = '/content/drive/My Drive/training_checkpoints'
  checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
  checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

  model.compile(optimizer='adam', loss=loss)
  history = model.fit_generator(df.data_generator(),epochs=EPOCHS,steps_per_epoch=1 ,callbacks=[checkpoint_callback])
  return model , df.tokenizer,df.idx2word

In [26]:
if __name__=="__main__":

  model_prediction ,tokenizer ,idx2word = train_model(max_vocab =2101 ,sequence_length = 256)
  pred = prediction(model_prediction, tokenizer , idx2word)
  next_char = input("enter the keyword")
  result = [next_char]
  for n in range(20):
    next_char = pred.pred_next_word(next_char)
    result.append(next_char)
    result.append(" ")

  result = tf.strings.join(result)
  print(result.numpy().decode('utf-8'), '\n\n' + '_'*80)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: 7.6497
Epoch 2/5
1/1 [==============================] - 0s 46ms/step - loss: 7.6502
Epoch 3/5
1/1 [==============================] - 0s 48ms/step - loss: 7.6506
Epoch 4/5
1/1 [==============================] - 0s 45ms/step - loss: 7.6502
Epoch 5/5
1/1 [==============================] - 0s 45ms/step - loss: 7.6515
enter the keywordnjan
njanenthoru pulkidumini manase tharaam viriyunnorishlormmakal eenangal ulakin kadalinte maunavum kondille kavaruthirunnundu choodumo kinnarathinum kudamoru ozhukiyoo… pulleem poy eeran nadhiyaay velayil  

________________________________________________________________________________
